In [1]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
import sys, os
print(sys.executable)
print(os.environ['PATH'])

/home/chen.ming1/work/6140/env/miniconda3/envs/cuda_env/bin/python
/home/chen.ming1/work/6140/env/miniconda3/envs/cuda_env/bin:/home/chen.ming1/work/6140/env/miniconda3/condabin:/home/chen.ming1/.vscode-server/bin/1a5daa3a0231a0fbba4f14db7ec463cf99d7768e/bin/remote-cli:/shared/centos7/discovery/bin:/usr/lib64/qt-3.3/bin:/usr/local/bin:/usr/bin:/usr/local/sbin:/usr/sbin:/home/chen.ming1/.local/bin:/home/chen.ming1/bin:/home/chen.ming1/.local/bin:/home/chen.ming1/bin


In [2]:
torch.cuda.is_available()

False

In [3]:
# Load the dataset
from transformers import TrainingArguments
from trl import SFTTrainer
from datasets import load_dataset
import logging

# Set up logging
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO
)


/home/chen.ming1/work/6140/env/miniconda3/envs/cuda_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/chen.ming1/work/6140/env/miniconda3/envs/cuda_env/lib/python3.9/site-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


In [5]:
dataset_name = "timdettmers/openassistant-guanaco"
dataset = load_dataset(dataset_name)
train_dataset, test_dataset = dataset["train"], dataset["test"]

In [4]:
# Load the model
from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer

class args:
    model_name_or_path = "EleutherAI/gpt-neo-1.3B"
    cache_dir = "./cache/"
    model_revision = "main"
    use_fast_tokenizer = True

config = AutoConfig.from_pretrained(
    args.model_name_or_path,
    cache_dir=args.cache_dir,
    revision=args.model_revision,
    use_auth_token=None,
)
tokenizer = AutoTokenizer.from_pretrained(
    args.model_name_or_path,
    cache_dir=args.cache_dir,
    use_fast=args.use_fast_tokenizer,
    revision=args.model_revision,
    use_auth_token=None,
)
model = AutoModelForCausalLM.from_pretrained(
    args.model_name_or_path,
    from_tf=bool(".ckpt" in args.model_name_or_path),
    config=config,
    cache_dir=args.cache_dir,
    revision=args.model_revision,
    use_auth_token=None,
)
tokenizer.pad_token = tokenizer.eos_token
model = model.to(device)

config.json: 100%|██████████| 1.35k/1.35k [00:00<00:00, 80.7kB/s]
tokenizer_config.json: 100%|██████████| 200/200 [00:00<00:00, 12.8kB/s]
vocab.json: 100%|██████████| 798k/798k [00:00<00:00, 22.9MB/s]
merges.txt: 100%|██████████| 456k/456k [00:00<00:00, 101MB/s]
special_tokens_map.json: 100%|██████████| 90.0/90.0 [00:00<00:00, 35.8kB/s]
model.safetensors: 100%|██████████| 5.31G/5.31G [01:53<00:00, 46.9MB/s]


KeyboardInterrupt: 

In [ ]:
#Use PEFT
from peft import LoraConfig, get_peft_model

class training_args:
    lora_rank = 4
    lora_alpha = 32

lora_config = LoraConfig(
    r=training_args.lora_rank,
    lora_alpha=training_args.lora_alpha,
    target_modules=["attn.attention.q_proj", "attn.attention.v_proj"],
    lora_dropout=0.1,
    bias="lora_only",
    modules_to_save=[],
)
model.enable_input_require_grads()
model = get_peft_model(model, lora_config)

In [ ]:
# Initialize the trainer
# Customize training arguments
output_dir = "./1.3b-all"
per_device_train_batch_size = 4
gradient_accumulation_steps = 4
optim = "adamw_torch"
save_steps = 10
logging_steps = 10
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 500
warmup_ratio = 0.03
lr_scheduler_type = "constant"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    half_precision_backend=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
    remove_unused_columns=False,
    resume_from_checkpoint=True,
    use_mps_device=True,
    # gradient_checkpointing=True,
    # evaluation_strategy="epoch",
)

# Initialize trainer
max_seq_length = 512
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)


In [ ]:
# Train the model
trainer.train()

pt_save_directory = "./test_all_1_3b"
tokenizer.save_pretrained(pt_save_directory)
model.save_pretrained(pt_save_directory)